In [1]:
import boto3
from datetime import date
import time
import uuid
from credentials import *
import unittest

## Connect to DynamoDB

Initially, we connect to our local database we started as "dynamodb" container:

In [2]:
client = boto3.client('dynamodb', 
  endpoint_url='http://dynamo:8000', 
  region_name='eu-central-1',
  aws_access_key_id=aws_access_key_id,
  aws_secret_access_key=aws_secret_access_key
)
client.list_tables()

{'TableNames': ['twitter'],
 'ResponseMetadata': {'RequestId': '2e6236b8-7eca-4059-8c1b-50f7903bc034',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 12 Jan 2024 14:34:44 GMT',
   'x-amzn-requestid': '2e6236b8-7eca-4059-8c1b-50f7903bc034',
   'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '2569046870',
   'content-length': '26',
   'server': 'Jetty(11.0.17)'},
  'RetryAttempts': 0}}

## Create Table

Let's setup the table with our attributes and indexes:

In [3]:
def create_app_table():
    return client.create_table(
        TableName="twitter",
        KeySchema=[
            {"AttributeName": "PK", "KeyType": "HASH"},
            {"AttributeName": "SK", "KeyType": "RANGE"},
        ],
        AttributeDefinitions=[
            {"AttributeName": "PK", "AttributeType": "S"},
            {"AttributeName": "SK", "AttributeType": "S"},
        ],
        ProvisionedThroughput={"ReadCapacityUnits": 10, "WriteCapacityUnits": 10},
    )

In [4]:
def delete_app_table():
    try:
        return client.delete_table(TableName="twitter")
    except:
        return False

In [5]:
delete_app_table()
create_app_table()

{'TableDescription': {'AttributeDefinitions': [{'AttributeName': 'PK',
    'AttributeType': 'S'},
   {'AttributeName': 'SK', 'AttributeType': 'S'}],
  'TableName': 'twitter',
  'KeySchema': [{'AttributeName': 'PK', 'KeyType': 'HASH'},
   {'AttributeName': 'SK', 'KeyType': 'RANGE'}],
  'TableStatus': 'ACTIVE',
  'CreationDateTime': datetime.datetime(2024, 1, 12, 14, 34, 44, 740000, tzinfo=tzlocal()),
  'ProvisionedThroughput': {'LastIncreaseDateTime': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=tzlocal()),
   'LastDecreaseDateTime': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=tzlocal()),
   'NumberOfDecreasesToday': 0,
   'ReadCapacityUnits': 10,
   'WriteCapacityUnits': 10},
  'TableSizeBytes': 0,
  'ItemCount': 0,
  'TableArn': 'arn:aws:dynamodb:ddblocal:000000000000:table/twitter',
  'DeletionProtectionEnabled': False},
 'ResponseMetadata': {'RequestId': '195d56ce-2373-4cb2-96e7-ed126bd37e5d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 12 Jan 2024 14:34:44 GMT',
   'x-amz

## Helpers

To setup some other keys later on, we create a function to extract the id from a key:

In [6]:
def parse_id_from_key(key):
    return key.split("#")[-1]

parse_id_from_key("CHAN#7")

'7'

## Users

We create and save our users:

In [7]:
uuid_entry1 = str(uuid.uuid4())
user_martin = {
    'PK': {'S': f"USER#{uuid_entry1}"},
    'SK': {'S': f"USER#{uuid_entry1}"},
    'Name': {'S': 'Martin Marsal'},
}

uuid_entry2 = str(uuid.uuid4())
user_christian = {
    'PK': {'S': f"USER#{uuid_entry2}"},
    'SK': {'S': f"USER#{uuid_entry2}"},
    'Name': {'S': 'Christian Diegmann'},
}

uuid_entry3 = str(uuid.uuid4())
user_robin = {
    'PK': {'S': f"USER#{uuid_entry3}"},
    'SK': {'S': f"USER#{uuid_entry3}"},
    'Name': {'S': 'Robin Schüle'},
}

def save_user(user):
    return client.put_item(
        TableName="twitter",
        Item=user
    )

# AP6
def find_user(primaryKey, sortKey):
    item = client.get_item(
        TableName="twitter",
        Key={
          'PK': { 'S': primaryKey },
          'SK': { 'S': sortKey }
        }
    )
    return item['Item'] if 'Item' in item else False

In [8]:
save_user(user_martin)
find_user(user_martin['PK']['S'], user_martin['SK']['S'])

{'SK': {'S': 'USER#8d633777-5926-4ef4-9210-ece14c3c4b37'},
 'PK': {'S': 'USER#8d633777-5926-4ef4-9210-ece14c3c4b37'},
 'Name': {'S': 'Martin Marsal'}}

In [9]:
save_user(user_christian)
find_user(user_christian['PK']['S'], user_christian['SK']['S'])

{'SK': {'S': 'USER#8f024649-4fcf-4171-8511-4c86930620ff'},
 'PK': {'S': 'USER#8f024649-4fcf-4171-8511-4c86930620ff'},
 'Name': {'S': 'Christian Diegmann'}}

In [10]:
save_user(user_robin)
find_user(user_robin['PK']['S'], user_robin['SK']['S'])

{'SK': {'S': 'USER#0f13e74d-bbf6-4188-be44-652a779cd63f'},
 'PK': {'S': 'USER#0f13e74d-bbf6-4188-be44-652a779cd63f'},
 'Name': {'S': 'Robin Schüle'}}

In [11]:
def add_to_followers(primaryKey, follower):
    response = client.put_item(
        TableName="twitter",
        Item={
            'PK': {'S': primaryKey},
            'SK': {'S': f"FOLLOWER#{parse_id_from_key(follower['PK']['S'])}"},
            'Name': {'S': follower['Name']['S']},
        },
        )
    if response['ResponseMetadata']['HTTPStatusCode'] == 200:
        return client.get_item(
            TableName='twitter',
            Key={
                'PK': {'S': primaryKey},
                'SK': {'S': f"FOLLOWER#{parse_id_from_key(follower['PK']['S'])}"},
            }
        )

In [12]:
add_to_followers(user_martin['PK']['S'], user_christian)

{'Item': {'SK': {'S': 'FOLLOWER#8f024649-4fcf-4171-8511-4c86930620ff'},
  'PK': {'S': 'USER#8d633777-5926-4ef4-9210-ece14c3c4b37'},
  'Name': {'S': 'Christian Diegmann'}},
 'ResponseMetadata': {'RequestId': '949a088a-b0c4-4d05-ba91-cc858ad69756',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 12 Jan 2024 14:34:46 GMT',
   'x-amzn-requestid': '949a088a-b0c4-4d05-ba91-cc858ad69756',
   'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '1469318709',
   'content-length': '158',
   'server': 'Jetty(11.0.17)'},
  'RetryAttempts': 0}}

In [13]:
add_to_followers(user_christian['PK']['S'], user_robin)

{'Item': {'SK': {'S': 'FOLLOWER#0f13e74d-bbf6-4188-be44-652a779cd63f'},
  'PK': {'S': 'USER#8f024649-4fcf-4171-8511-4c86930620ff'},
  'Name': {'S': 'Robin Schüle'}},
 'ResponseMetadata': {'RequestId': '861e91f0-9870-4c78-aadd-9f1f93d4b7c0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 12 Jan 2024 14:34:46 GMT',
   'x-amzn-requestid': '861e91f0-9870-4c78-aadd-9f1f93d4b7c0',
   'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '1569167597',
   'content-length': '153',
   'server': 'Jetty(11.0.17)'},
  'RetryAttempts': 0}}

In [14]:
add_to_followers(user_robin['PK']['S'], user_martin)

{'Item': {'SK': {'S': 'FOLLOWER#8d633777-5926-4ef4-9210-ece14c3c4b37'},
  'PK': {'S': 'USER#0f13e74d-bbf6-4188-be44-652a779cd63f'},
  'Name': {'S': 'Martin Marsal'}},
 'ResponseMetadata': {'RequestId': 'f6acf799-499e-46db-ab8b-7e8457bbc071',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 12 Jan 2024 14:34:46 GMT',
   'x-amzn-requestid': 'f6acf799-499e-46db-ab8b-7e8457bbc071',
   'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '3398539310',
   'content-length': '153',
   'server': 'Jetty(11.0.17)'},
  'RetryAttempts': 0}}

## 1st access pattern: Post a tweet

In [15]:
uuid_entry4 = str(uuid.uuid4())
timestamp = int(time.time())
tweet = {'id': {'S': f"{uuid_entry4}"}, 'text': {'S': 'Moin, moin.'}, 'likes': {'N': '0'}, 'CreatedAt': { 'N': str(timestamp)}}
uuid_entry5 = str(uuid.uuid4())
timestamp = int(time.time())
tweet_2 = {'id': {'S': f"{uuid_entry5}"}, 'text': {'S': 'MongoDB ist super!'}, 'likes': {'N': '0'}, 'CreatedAt': { 'N': str(timestamp)}}
newMartin = find_user(user_martin['PK']['S'], user_martin['SK']['S'])

def post_tweet(primaryKey, tweet):
    client.put_item(
        TableName='twitter',
        Item={
            'PK': {'S': primaryKey},
            'SK': {'S': f"TWEET#{tweet['id']['S']}"},
            'id': {'S': tweet['id']['S']},
            'text': {'S': tweet['text']['S']},
            'likes': {'N': tweet['likes']['N']},
            'CreatedAt': { 'N': tweet['CreatedAt']['N']}
        },
    )
    item = client.query(
        TableName="twitter",
        KeyConditionExpression='PK = :pk AND begins_with(SK, :follower)',
        ExpressionAttributeValues={
            ':pk': { 'S': primaryKey },
            ':follower': { 'S': 'FOLLOWER#' }
        },
    )
    followers = item['Items']
    for follower in followers:
        response2 = client.put_item(
            TableName='twitter',
            Item={
                'PK': {'S': f"USER#{parse_id_from_key(follower['SK']['S'])}"},
                'SK': {'S': f"TIMELINE#{tweet['id']['S']}"},
                'tweetId': {'S': tweet['id']['S']},
                'text': {'S': tweet['text']['S']},
                'likes': {'N': tweet['likes']['N']},
                'CreatedAt': { 'N': tweet['CreatedAt']['N']}
            },
        )
    if response2['ResponseMetadata']['HTTPStatusCode'] == 200:
        return client.query(
            TableName="twitter",
            KeyConditionExpression='PK = :pk AND begins_with(SK, :timeline)',
            ExpressionAttributeValues={
                ':pk': { 'S': f"USER#{parse_id_from_key(user_christian['PK']['S'])}" },
                ':timeline': { 'S': 'TIMELINE#' }
            },
        )

In [16]:
post_tweet(user_martin['PK']['S'], tweet)

{'Items': [{'SK': {'S': 'TIMELINE#9a537d82-f535-4a92-a5c4-6b0bb7754dfc'},
   'PK': {'S': 'USER#8f024649-4fcf-4171-8511-4c86930620ff'},
   'text': {'S': 'Moin, moin.'},
   'tweetId': {'S': '9a537d82-f535-4a92-a5c4-6b0bb7754dfc'},
   'likes': {'N': '0'},
   'CreatedAt': {'N': '1705070086'}}],
 'Count': 1,
 'ScannedCount': 1,
 'ResponseMetadata': {'RequestId': '531b84ff-af52-40aa-bb08-be2249784377',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 12 Jan 2024 14:34:47 GMT',
   'x-amzn-requestid': '531b84ff-af52-40aa-bb08-be2249784377',
   'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '1588103398',
   'content-length': '285',
   'server': 'Jetty(11.0.17)'},
  'RetryAttempts': 0}}

In [17]:
post_tweet(user_martin['PK']['S'], tweet_2)

{'Items': [{'SK': {'S': 'TIMELINE#9a537d82-f535-4a92-a5c4-6b0bb7754dfc'},
   'PK': {'S': 'USER#8f024649-4fcf-4171-8511-4c86930620ff'},
   'text': {'S': 'Moin, moin.'},
   'tweetId': {'S': '9a537d82-f535-4a92-a5c4-6b0bb7754dfc'},
   'likes': {'N': '0'},
   'CreatedAt': {'N': '1705070086'}},
  {'SK': {'S': 'TIMELINE#b2edf91f-83a5-42f2-9323-75b5b735e1b9'},
   'PK': {'S': 'USER#8f024649-4fcf-4171-8511-4c86930620ff'},
   'text': {'S': 'MongoDB ist super!'},
   'tweetId': {'S': 'b2edf91f-83a5-42f2-9323-75b5b735e1b9'},
   'likes': {'N': '0'},
   'CreatedAt': {'N': '1705070086'}}],
 'Count': 2,
 'ScannedCount': 2,
 'ResponseMetadata': {'RequestId': 'f00fd85a-3bd1-4437-b86f-0f3be269c28e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 12 Jan 2024 14:34:47 GMT',
   'x-amzn-requestid': 'f00fd85a-3bd1-4437-b86f-0f3be269c28e',
   'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '692872706',
   'content-length': '539',
   'server': 'Jetty(11.0.17)'},
  'RetryAttempts': 0}}

In [18]:
class TestPostTweet(unittest.TestCase):
    def test_post_tweet_enriched_timeline(self):
        christian_timeline = client.query(
            TableName="twitter",
            KeyConditionExpression='PK = :pk AND begins_with(SK, :timeline)',
            ExpressionAttributeValues={
                ':pk': { 'S': user_christian['PK']['S'] },
                ':timeline': { 'S': 'TIMELINE#' }
            },
        )
        timeline_length = len(christian_timeline['Items'])
        self.assertEqual(timeline_length, 2)

    def test_post_tweet_empty_timeline(self):
        robin_timeline = client.query(
            TableName="twitter",
            KeyConditionExpression='PK = :pk AND begins_with(SK, :timeline)',
            ExpressionAttributeValues={
                ':pk': { 'S': user_robin['PK']['S'] },
                ':timeline': { 'S': 'TIMELINE#' }
            },
        )
        timeline_length = len(robin_timeline['Items'])
        self.assertEqual(timeline_length, 0)

In [19]:
if __name__ == '__main__':
    suite = unittest.TestLoader().loadTestsFromTestCase(TestPostTweet)
    unittest.TextTestRunner(verbosity=2).run(suite)

test_post_tweet_empty_timeline (__main__.TestPostTweet.test_post_tweet_empty_timeline) ... ok
test_post_tweet_enriched_timeline (__main__.TestPostTweet.test_post_tweet_enriched_timeline) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.048s

OK


## 2nd access pattern: Write a reply

In [20]:
uuid_entry6 = str(uuid.uuid4())
reply = {'id': {'S': f"{uuid_entry6}"}, 'text': {'S': 'Hallo zurück.'}, 'likes': {'N': '0'}, 'CreatedAt': { 'N': str(timestamp)}, 'tweetId': {'S': f"{parse_id_from_key(tweet['id']['S'])}"},}

def post_reply(author, user, tweet, reply):
    client.put_item(
        TableName='twitter',
        Item={
            'PK': {'S': f"USER#{parse_id_from_key(author['PK']['S'])}"},
            'SK': {'S': f"TWEET#{tweet['id']['S']}#{reply['id']['S']}"},
            'id': {'S': reply['id']['S']},
            'text': {'S': reply['text']['S']},
            'likes': {'N': reply['likes']['N']},
            'CreatedAt': { 'N': reply['CreatedAt']['N']}
        },
    )
    item = client.query(
        TableName="twitter",
        KeyConditionExpression='PK = :pk AND begins_with(SK, :follower)',
        ExpressionAttributeValues={
            ':pk': { 'S': f"USER#{parse_id_from_key(author['PK']['S'])}" },
            ':follower': { 'S': 'FOLLOWER#' }
        },
    )
    followers = item['Items']
    for follower in followers:
        response2 = client.put_item(
            TableName='twitter',
            Item={
                'PK': {'S': f"USER#{parse_id_from_key(follower['SK']['S'])}"},
                'SK': {'S': f"TIMELINE#{tweet['id']['S']}#{reply['id']['S']}"},
                'id': {'S': reply['id']['S']},
                'text': {'S': reply['text']['S']},
                'likes': {'N': reply['likes']['N']},
                'CreatedAt': { 'N': reply['CreatedAt']['N']}
            },
        )
    if response2['ResponseMetadata']['HTTPStatusCode'] == 200:
        return client.query(
            TableName="twitter",
            KeyConditionExpression='PK = :pk AND begins_with(SK, :timeline)',
            ExpressionAttributeValues={
                ':pk': { 'S': f"USER#{parse_id_from_key(author['PK']['S'])}" },
                ':timeline': { 'S': 'TIMELINE#' }
            },
        )

In [21]:
post_reply(user_martin, user_christian, tweet, reply)

{'Items': [],
 'Count': 0,
 'ScannedCount': 0,
 'ResponseMetadata': {'RequestId': '48d41019-7b4a-41ef-af0f-f71e7f106863',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 12 Jan 2024 14:34:48 GMT',
   'x-amzn-requestid': '48d41019-7b4a-41ef-af0f-f71e7f106863',
   'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '583912891',
   'content-length': '39',
   'server': 'Jetty(11.0.17)'},
  'RetryAttempts': 0}}

In [22]:
class TestPostReply(unittest.TestCase):
    def test_post_reply_enriched_replies(self):
        test_tweet = client.query(
            TableName="twitter",
            KeyConditionExpression='PK = :pk AND begins_with(SK, :timeline)',
            ExpressionAttributeValues={
                ':pk': { 'S': user_christian['PK']['S'] },
                ':timeline': { 'S': f"TIMELINE#{tweet['id']['S']}" }
            },
        )
        tweet_length = len(test_tweet['Items'])
        self.assertEqual(tweet_length, 2)
        
    def test_post_reply_empty_replies(self):
        test_tweet = client.query(
            TableName="twitter",
            KeyConditionExpression='PK = :pk AND begins_with(SK, :timeline)',
            ExpressionAttributeValues={
                ':pk': { 'S': user_christian['PK']['S'] },
                ':timeline': { 'S': f"TIMELINE#{tweet_2['id']['S']}" }
            },
        )
        tweet_length = len(test_tweet['Items'])
        self.assertEqual(tweet_length, 1)

In [23]:
if __name__ == '__main__':
    suite = unittest.TestLoader().loadTestsFromTestCase(TestPostReply)
    unittest.TextTestRunner(verbosity=2).run(suite)

test_post_reply_empty_replies (__main__.TestPostReply.test_post_reply_empty_replies) ... ok
test_post_reply_enriched_replies (__main__.TestPostReply.test_post_reply_enriched_replies) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.032s

OK


# 3rd access pattern: Edit a tweet

In [24]:
def edit_tweet(primaryKey, tweetId, new_text):

    response = client.update_item(
        TableName="twitter",
        Key={
            'PK': {'S': primaryKey},
            'SK': {'S': f"TWEET#{tweetId}"}
        },
        UpdateExpression='SET #text = :new_text',
        ExpressionAttributeNames={
            '#text': 'text'
        },
        ExpressionAttributeValues={
            ':new_text': {'S': new_text}
        }
    )


    followers_query = client.query(
        TableName="twitter",
        KeyConditionExpression='PK = :pk AND begins_with(SK, :follower)',
        ExpressionAttributeValues={
            ':pk': {'S': primaryKey},
            ':follower': {'S': 'FOLLOWER#'}
        },
    )

    followers = followers_query.get('Items', [])
    for follower in followers:
        client.update_item(
            TableName="twitter",
            Key={
                'PK': {'S': f"USER#{parse_id_from_key(follower['SK']['S'])}"},
                'SK': {'S': f"TIMELINE#{tweetId}"}
            },
            UpdateExpression='SET #text = :new_text',
            ExpressionAttributeNames={
                '#text': 'text'
            },
            ExpressionAttributeValues={
                ':new_text': {'S': new_text}
            }
        )

    return response

In [25]:
new_tweet_text = "DynamoDB ist super"
edit_tweet(user_martin['PK']['S'], uuid_entry5, new_tweet_text)

{'ResponseMetadata': {'RequestId': '7299a281-ae79-4292-93b5-dbb9415b8045',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 12 Jan 2024 14:34:49 GMT',
   'x-amzn-requestid': '7299a281-ae79-4292-93b5-dbb9415b8045',
   'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '2745614147',
   'content-length': '2',
   'server': 'Jetty(11.0.17)'},
  'RetryAttempts': 0}}

In [26]:
def get_user_tweets(primaryKey):
    response = client.query(
        TableName="twitter",
        KeyConditionExpression='PK = :pk AND begins_with(SK, :tweet)',
        ExpressionAttributeValues={
            ':pk': {'S': primaryKey},
            ':tweet': {'S': 'TWEET#'}
        }
    )

    tweets = response.get('Items', [])
    return tweets

In [27]:
user_martin_tweets = get_user_tweets(user_martin['PK']['S'])
print(user_martin_tweets)

[{'SK': {'S': 'TWEET#9a537d82-f535-4a92-a5c4-6b0bb7754dfc'}, 'PK': {'S': 'USER#8d633777-5926-4ef4-9210-ece14c3c4b37'}, 'id': {'S': '9a537d82-f535-4a92-a5c4-6b0bb7754dfc'}, 'text': {'S': 'Moin, moin.'}, 'likes': {'N': '0'}, 'CreatedAt': {'N': '1705070086'}}, {'SK': {'S': 'TWEET#9a537d82-f535-4a92-a5c4-6b0bb7754dfc#c98f7137-dae2-4e4b-ae14-09aebe1561f6'}, 'PK': {'S': 'USER#8d633777-5926-4ef4-9210-ece14c3c4b37'}, 'id': {'S': 'c98f7137-dae2-4e4b-ae14-09aebe1561f6'}, 'text': {'S': 'Hallo zurück.'}, 'likes': {'N': '0'}, 'CreatedAt': {'N': '1705070086'}}, {'SK': {'S': 'TWEET#b2edf91f-83a5-42f2-9323-75b5b735e1b9'}, 'PK': {'S': 'USER#8d633777-5926-4ef4-9210-ece14c3c4b37'}, 'id': {'S': 'b2edf91f-83a5-42f2-9323-75b5b735e1b9'}, 'text': {'S': 'DynamoDB ist super'}, 'CreatedAt': {'N': '1705070086'}, 'likes': {'N': '0'}}]


In [28]:
class TestEditTweet(unittest.TestCase):
    def test_edit_tweet__timeline(self):
        tweet_id_to_find = uuid_entry5  
        updated_text = "DynamoDB ist super"

        christian_timeline = client.query(
            TableName="twitter",
            KeyConditionExpression='PK = :pk AND SK = :sk',
            ExpressionAttributeValues={
                ':pk': {'S': f"USER#{parse_id_from_key(user_christian['PK']['S'])}"},
                ':sk': {'S': f"TIMELINE#{tweet_id_to_find}"}
            },
        )
        matching_items = christian_timeline.get('Items', [])

        self.assertTrue(
            any('text' in item and item['text']['S'] == updated_text for item in matching_items),
            f"Tweet with ID {tweet_id_to_find} not found in timeline or text not updated."
        )

    def test_edit_tweet_martin(self):
        tweet_id_to_find = uuid_entry5  
        updated_text = "DynamoDB ist super"  

        
        martin_tweets = client.query(
            TableName="twitter",
            KeyConditionExpression='PK = :pk AND SK = :sk',
            ExpressionAttributeValues={
                ':pk': {'S': f"USER#{parse_id_from_key(user_martin['PK']['S'])}"},
                ':sk': {'S': f"TWEET#{tweet_id_to_find}"}
            },
        )
        matching_items_martin = martin_tweets.get('Items', [])

        # Check if the updated text is found in user_martin's tweets
        self.assertTrue(
            any('text' in item and item['text']['S'] == updated_text for item in matching_items_martin),
            f"Tweet with ID {tweet_id_to_find} not found in Martin's tweets or text not updated."
        )


    def test_edit_tweet_empty_timeline(self):
        robin_timeline = client.query(
            TableName="twitter",
            KeyConditionExpression='PK = :pk AND begins_with(SK, :timeline)',
            ExpressionAttributeValues={
                ':pk': { 'S': user_robin['PK']['S'] },
                ':timeline': { 'S': 'TIMELINE#' }
            },
        )
        timeline_length = len(robin_timeline['Items'])
        self.assertEqual(timeline_length, 0)

In [29]:
if __name__ == '__main__':
    suite = unittest.TestLoader().loadTestsFromTestCase(TestEditTweet)
    unittest.TextTestRunner(verbosity=2).run(suite)


test_edit_tweet__timeline (__main__.TestEditTweet.test_edit_tweet__timeline) ... ok
test_edit_tweet_empty_timeline (__main__.TestEditTweet.test_edit_tweet_empty_timeline) ... ok
test_edit_tweet_martin (__main__.TestEditTweet.test_edit_tweet_martin) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.060s

OK


# 4th access pattern: Read a timeline

In [30]:
def read_timeline(primaryKey):
    timeline_query = client.query(
        TableName="twitter",
        KeyConditionExpression='PK = :pk AND begins_with(SK, :timeline)',
        ExpressionAttributeValues={
            ':pk': {'S': primaryKey},
            ':timeline': {'S': 'TIMELINE#'}
        },
    )
    timeline_items = timeline_query.get('Items', [])
    
    formatted_timeline = [
        {
            'id': item.get('tweetId', item.get('id', 'N/A')),
            'text': item.get('text', 'N/A'),
            'likes': item.get('likes', 'N/A'),
            'CreatedAt': item.get('CreatedAt', 'N/A')
        }
        for item in timeline_items
    ]


    return formatted_timeline

In [31]:
user_christian_timeline = read_timeline(user_christian['PK']['S'])
print(user_christian_timeline)

[{'id': {'S': '9a537d82-f535-4a92-a5c4-6b0bb7754dfc'}, 'text': {'S': 'Moin, moin.'}, 'likes': {'N': '0'}, 'CreatedAt': {'N': '1705070086'}}, {'id': {'S': 'c98f7137-dae2-4e4b-ae14-09aebe1561f6'}, 'text': {'S': 'Hallo zurück.'}, 'likes': {'N': '0'}, 'CreatedAt': {'N': '1705070086'}}, {'id': {'S': 'b2edf91f-83a5-42f2-9323-75b5b735e1b9'}, 'text': {'S': 'DynamoDB ist super'}, 'likes': {'N': '0'}, 'CreatedAt': {'N': '1705070086'}}]


In [32]:
class TestReadTimeline(unittest.TestCase):
    def test_read_timeline(self):
        # Assuming user_christian has a populated timeline
        expected_texts = ["Moin, moin.", "Hallo zurück.", "DynamoDB ist super"]

        # Fetch the timeline of user_christian
        christian_timeline = read_timeline(user_christian['PK']['S'])

        # Extract text items from the timeline
        actual_texts = [item['text']['S'] for item in christian_timeline]

        # Check if the actual text items match the expected texts (order doesn't matter)
        self.assertCountEqual(actual_texts, expected_texts, "Timeline text items do not match the expected texts.")

In [33]:
if __name__ == '__main__':
    suite = unittest.TestLoader().loadTestsFromTestCase(TestReadTimeline)
    unittest.TextTestRunner(verbosity=2).run(suite)

test_read_timeline (__main__.TestReadTimeline.test_read_timeline) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.018s

OK


# 5th access pattern: Delete a Tweet

In [34]:
def delete_tweet(primaryKey, tweet_id):
    # Löschen des Tweets aus der Tweet-Entität
    client.delete_item(
        TableName='twitter',
        Key={'PK': {'S': primaryKey}, 'SK': {'S': f"TWEET#{tweet_id}"}}
    )

    # Löschen des Tweets aus den Timelines aller Follower
    followers = client.query(
        TableName="twitter",
        KeyConditionExpression='PK = :pk AND begins_with(SK, :msg)',
        ExpressionAttributeValues={
            ':pk': {'S': primaryKey},
            ':msg': {'S': 'FOLLOWER#'}
        },
    )['Items']

    for follower in followers:
        client.delete_item(
            TableName='twitter',
            Key={'PK': {'S': follower['SK']['S']}, 'SK': {'S': f"TIMELINE#{tweet_id}"}}
        )


In [36]:
class TestDeleteTweet(unittest.TestCase):

    def test_delete_tweet(self):
        user_id = uuid_entry1
        tweet_id = uuid_entry5
        user_key = f'USER#{user_id}'
        tweet_key = f'TWEET#{tweet_id}'

        delete_tweet(user_id, tweet_id)

        tweet_item = client.get_item(TableName='twitter', Key={'PK': {'S': tweet_key}, 'SK': {'S': tweet_key}})
        self.assertNotIn('Item', tweet_item)

In [37]:
if __name__ == '__main__':
    suite = unittest.TestLoader().loadTestsFromTestCase(TestDeleteTweet)
    unittest.TextTestRunner(verbosity=2).run(suite)

test_delete_tweet (__main__.TestDeleteTweet.test_delete_tweet) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.054s

OK


# 6th access pattern: Delete a User

In [38]:
def delete_user(user_id):
    user_tweets_response = client.query(
        TableName='twitter',
        KeyConditionExpression='#pk = :pk',
        ExpressionAttributeNames={'#pk': 'PK'},
        ExpressionAttributeValues={':pk': {'S': f'USER#{user_id}'}}
    )

    user_tweets = user_tweets_response.get('Items', [])

    for tweet in user_tweets:
        tweet_id = tweet.get('id', {}).get('S')  # Zugriff auf das tatsächlich vorhandene Feld 'id'

        if tweet_id:
            client.delete_item(
                TableName='twitter',
                Key={'PK': {'S': tweet['PK']['S']}, 'SK': {'S': tweet['SK']['S']}}
            )

            followers_response = client.query(
                TableName='twitter',
                KeyConditionExpression='#pk = :pk',
                ExpressionAttributeNames={'#pk': 'PK'},
                ExpressionAttributeValues={':pk': {'S': tweet['PK']['S']}}
            )

            for follower in followers_response.get('Items', []):
                client.delete_item(
                    TableName='twitter',
                    Key={'PK': {'S': follower['SK']['S']}, 'SK': {'S': f'TIMELINE#{tweet_id}'}}
                )

    client.delete_item(
        TableName='twitter',
        Key={'PK': {'S': f'USER#{user_id}'}, 'SK': {'S': f'USER#{user_id}'}}
    )

In [39]:
class TestDeleteUser(unittest.TestCase):
    
    def test_user_deleted(self):
        user_id = uuid_entry1
        user_key = f'USER#{user_id}'

        # Funktion aufrufen
        delete_user(user_id)

        # Überprüfen, ob der Benutzer und der Tweet erfolgreich gelöscht wurden
        user_item = client.get_item(TableName='twitter', Key={'PK': {'S': user_key}, 'SK': {'S': user_key}})
        self.assertNotIn('Item', user_item)

    def test_tweets_deleted(self):
        tweet_id = uuid_entry4
        tweet_key = f'TWEET#{tweet_id}'
        
        tweet_item = client.get_item(TableName='twitter', Key={'PK': {'S': tweet_key}, 'SK': {'S': tweet_key}})
        self.assertNotIn('Item', tweet_item)
        

In [40]:
if __name__ == '__main__':
    suite = unittest.TestLoader().loadTestsFromTestCase(TestDeleteUser)
    unittest.TextTestRunner(verbosity=2).run(suite)

test_tweets_deleted (__main__.TestDeleteUser.test_tweets_deleted) ... ok
test_user_deleted (__main__.TestDeleteUser.test_user_deleted) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.364s

OK
